## Part1. 분류(Classification)

### 1. 분석 데이터 준비

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('../breast-cancer-wisconsin.csv')
data.head()

,code,Clump_Thickness,Cell_Size,Cell_Shape,Marginal_Adhesion,Single_Epithelial_Cell_Size,Bare_Nuclei,Bland_Chromatin,Normal_Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,0
1,1002945,5,4,4,5,7,10,3,2,1,0
2,1015425,3,1,1,1,2,2,3,1,1,0
3,1016277,6,8,8,1,3,4,3,7,1,0
4,1017023,4,1,1,3,2,1,3,1,1,0


In [14]:
X=data[data.columns[1:10]]
y=data[['Class']]

In [15]:
# 학습, 테스트 데이터 셋 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,stratify=y, random_state=42)

In [19]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

---

### 2. 기본 모델 적용

In [20]:
from sklearn.neighbors import KNeighborsClassifier

In [23]:
model=KNeighborsClassifier()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.984375

In [27]:
# 혼동헹렬
from sklearn.metrics import confusion_matrix
confusion_train = confusion_matrix(y_train, pred_train)
print('훈련데이터 오차행렬: \n',confusion_train)

훈련데이터 오차행렬: 
 [[331   2]
 [  6 173]]


In [30]:
# 예측 레포트
from sklearn.metrics import classification_report
cfreport_train = classification_report(y_train, pred_train)
print('분류 예측 레포트:\n', cfreport_train)

분류 예측 레포트:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99       333
           1       0.99      0.97      0.98       179

    accuracy                           0.98       512
   macro avg       0.99      0.98      0.98       512
weighted avg       0.98      0.98      0.98       512



In [33]:
# 테스트데이터
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.9532163742690059

In [35]:
confusion_test = confusion_matrix(y_test, pred_test)
print('테스트데이터 오차행렬:\n', confusion_test)

테스트데이터 오차행렬:
 [[106   5]
 [  3  57]]


In [36]:
cfreport_test=classification_report(y_test, pred_test)
print("분류예측 레포트:\n", cfreport_test)

분류예측 레포트:
               precision    recall  f1-score   support

           0       0.97      0.95      0.96       111
           1       0.92      0.95      0.93        60

    accuracy                           0.95       171
   macro avg       0.95      0.95      0.95       171
weighted avg       0.95      0.95      0.95       171



### 3. Grid Search

In [37]:
from sklearn.model_selection import GridSearchCV

In [38]:
param = {'n_neighbors':[1,3,5,7,9,11]}

In [41]:
grid_search = GridSearchCV(KNeighborsClassifier(), param, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9, 11]})

In [45]:
print(f'Best Param: {grid_search.best_params_}')
print(f'Best Score: {round(grid_search.best_score_,3)}')
print(f'TestSet Score: {round(grid_search.score(X_scaled_test, y_test),3)}')

Best Param: {'n_neighbors': 3}
Best Score: 0.982
TestSet Score: 0.953


### 4. Random Search

In [46]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

In [47]:
param_dist = {'n_neighbors' : randint(low=1, high=20)}
random_search = RandomizedSearchCV(KNeighborsClassifier(), param_dist, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(),
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000203C7C47D90>})

In [50]:
print(f"Best Param: {random_search.best_params_}")
print(f"Best Score: {round(random_search.best_score_,2)}")
print(f"Test set Score : {round(random_search.score(X_scaled_test, y_test),3)}")

Best Param: {'n_neighbors': 3}
Best Score: 0.98
Test set Score : 0.953


---

## Part2. 회귀(Regression)

### 1.분석 데이터 준비

In [51]:
data1 = pd.read_csv('../house_price.csv')
data1.head()

,housing_age,income,bedrooms,households,rooms,house_value
0,23,6.7770,0.141112,2.442244,8.103960,500000
1,49,6.0199,0.160984,2.726688,5.752412,500000
2,35,5.1155,0.249061,1.902676,3.888078,500000
3,32,4.7109,0.231383,1.913669,4.508393,500000
4,21,4.5625,0.255583,3.092664,4.667954,500000


In [55]:
# 특성 데이터 나누기
X = data1[data1.columns[1:5]]
y = data1[['house_value']]

In [56]:
# 훈련, 테스트 데이터 나누기
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, random_state=42)

In [58]:
# 정규화
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train)
X_scaled_train = scaler.transform(X_train)
X_scaled_test = scaler.transform(X_test)

### 2. 기본모델 적용

In [61]:
from sklearn.neighbors import KNeighborsRegressor

In [63]:
model = KNeighborsRegressor()
model.fit(X_scaled_train, y_train)
pred_train = model.predict(X_scaled_train)
model.score(X_scaled_train, y_train)

0.6804607237174459

In [64]:
# 테스트 데이터
pred_test = model.predict(X_scaled_test)
model.score(X_scaled_test, y_test)

0.5541889571372401

In [65]:
# RMSE (Root Mean Squared Error)
import numpy as np
from sklearn.metrics import mean_squared_error

In [70]:
MSE_train = mean_squared_error(y_train, pred_train)
MSE_test = mean_squared_error(y_test, pred_test)
print('훈련 데이터 RMSE: ',np.sqrt(MSE_train))
print('테스트 데이터 RMSE: ',np.sqrt(MSE_test))

훈련 데이터 RMSE:  53952.69804097723
테스트 데이터 RMSE:  63831.91662964773


### 3.Grid Search

In [71]:
from sklearn.model_selection import GridSearchCV

In [74]:
param = {'n_neighbors' : [1,3,5,7,9]}
grid_search = GridSearchCV(KNeighborsRegressor(), param, cv=5)
grid_search.fit(X_scaled_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 3, 5, 7, 9]})

In [77]:
print(f'Best param : {grid_search.best_params_}')
print(f'Best Score: {round(grid_search.best_score_,3)}')
print("TestSet Score: {:.4f}".format(grid_search.score(X_scaled_test, y_test)))

Best param : {'n_neighbors': 9}
Best Score: 0.559
TestSet Score: 0.5798


### 4. Random Search

In [78]:
from sklearn.model_selection import RandomizedSearchCV

In [79]:
param_distribs = {'n_neighbors': randint(low=1, high=20)}
random_search=RandomizedSearchCV(KNeighborsRegressor(), 
                                 param_distributions=param_distribs, n_iter=50, cv=5)
random_search.fit(X_scaled_train, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsRegressor(), n_iter=50,
                   param_distributions={'n_neighbors': <scipy.stats._distn_infrastructure.rv_frozen object at 0x00000203C7CC0460>})

In [80]:
print("Best Parameter: {}".format(random_search.best_params_))
print("Best Score: {:.4f}".format(random_search.best_score_))
print("TestSet Score: {:.4f}".format(random_search.score(X_scaled_test, y_test)))

Best Parameter: {'n_neighbors': 18}
Best Score: 0.5769
TestSet Score: 0.5992
